# Importações

In [33]:
from selenium import webdriver
import mysql.connector
from mysql.connector import errorcode
import os

# Funções

### conexão MySQL

In [34]:
def conecta():
    try:
        global conexao
        conexao = mysql.connector.connect(
          host="localhost", #ip
          user="root", #usuario
          password="123456", #senha
          database="inpi" #banco
        )
        global cursor
        cursor = conexao.cursor()

    except mysql.connector.Error as error:
        if error.errno == errorcode.ER_BAD_DB_ERROR:
            print("Banco de dados não existe")
        elif error.errno == errorcode.ER_ACCESS_DENIED_ERROR:
            print("Usuario ou senha incorretos")
        else:
            print(error)
        return False
    else:
        print("Conexão realizada com sucesso")
        return True

### Recupera uma faixa para trabalhar
A = Aberta /
P = Processando /
F = Finalizada /
M = Em manutenção

In [35]:
def faixa():
    sql = "select Id, Inicio, Fim, Progresso from tb_faixas where Status = 'A' LIMIT 1"
    cursor.execute(sql)
    fx = cursor.fetchone()

    global idFaixa
    idFaixa = fx[0]
    global inicio
    inicio = fx[1]
    global total
    total = fx[2]
    global progresso
    progresso = fx[3]
    global diretorio
    diretorio = 'Fotos/Faixa '+str(idFaixa)
    if not os.path.exists(diretorio):
        os.mkdir(diretorio)

### Atualiza faixa para não haver conflitos

In [36]:
def attFaixa():
    sql = "update tb_faixas set Status = 'P' where Id = " + str(idFaixa)
    cursor.execute(sql)

    conexao.commit()


### Configura driver e inicia

In [37]:
def browser():
    options = webdriver.ChromeOptions()
    options.add_argument('ignore-certificate-errors')
    global driver
    driver = webdriver.Chrome(chrome_options=options)


### Inicia processo de autenticação para inicio da mineração


In [38]:
def autentica():
    driver.get("https://busca.inpi.gov.br/pePI/servlet/LoginController?action=login")
    driver.get("https://busca.inpi.gov.br/pePI/jsp/marcas/Pesquisa_classe_basica.jsp")
    input_marca = driver.find_element_by_name("marca")
    input_marca.send_keys("teste")

    submit_button = driver.find_element_by_name("botao")
    submit_button.click()

## Funções de Captura de dados

### Dados da Marca

In [39]:
def dados_marca(count):

    try:
        tr = driver.find_elements_by_xpath("//table[@border='0']//font")

        n_processo = tr[1].text
        marca = tr[3].text
        situacao = tr[5].text
        apresentacao = tr[7].text
        natureza = tr[9].text

        #captura imagem
        try:
            img = "{}/{}.png".format(diretorio, n_processo)
            with open(img, 'wb') as file:
                    file.write(driver.find_element_by_xpath("//img[contains(@src,'LogoMarcasServletController')]").screenshot_as_png)
        except:
            img = None
    except:
        n_processo = None
        marca = None
        situacao = None
        apresentacao = None
        natureza = None
        img = None

    sql = "INSERT INTO tb_marca (Id, Processo, Marca, Situacao, Apresentacao, Natureza, Foto) VALUES (%s, %s, %s, %s, %s, %s, %s)"
    val = (count, n_processo, marca, situacao, apresentacao, natureza, img)
    cursor.execute(sql, val)

    conexao.commit()

# Mineração

In [40]:
if conecta():

    faixa()
    attFaixa()
    browser()
    autentica()

    for n in range(progresso, total):
        uri = "https://busca.inpi.gov.br/pePI/servlet/MarcasServletController?Action=detail&CodPedido=" + str(n)
        driver.get(uri)
        dados_marca(n)
        exit()
#

Conexão realizada com sucesso


<ipython-input-37-fe5f3451e6d1>:5: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=options)


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//img[contains(@src,'LogoMarcasServletController')]"}
  (Session info: chrome=89.0.4389.114)
